# Project Basics Of Mobile Robotics

### Computer Vision


### Global navigation

# Bayesian Filter
This section has been written and the code made by Jean Cordonnier
## Introduction
We decided to go with a kalman filter as our bayesian filter. Our choice has been made based on 2 factors: it does not take big computational power (like the particle filter) and we assumed gaussian noise on the system. To implement the filter we used a python library from GitHub : https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python/. The installation is in requirement.txt. 

To deal with the non-linearities in the orientation and position (we needed to change frame using the angle of the robot for the acceleration ad the velocity) and to deal with the format of the measurement (position ,velocity and acceleration being 2 dimensions variables and heading and spin (dheading/dt) being one dimensionnal) we decided to go with two kalmans: One for the position and one for the orientation. We will refer to them as kf_pos and kf_rot. Both are implemented in a kalman class defined in kalman.py
## Kalman for Position implementation
### Choice for measurement and state variables
For our measurements we used the data from the accelerometers, the average of the velocity of the wheels and the position from the camera. 
The kalman coordinates are exprimed in the local frame (with 0,0 being the top left corner) and the velocity and the accelerometers were exprimed in the body frame (centered with the robot) we had to do a coordinate change represented by this function: 
```python
#Change from body frame to local frame
def change_frame(body_data,body_orientation):
    '''
    Change from body frame to local frame.

    Parameters:
    - body_data: Data expressed in the body frame.
    - body_orientation: Orientation of the robot in the local frame (from kf_rot).

    Returns:
    Data expressed in the local frame.
    '''
    R = np.array([[np.cos(body_orientation), -np.sin(body_orientation)],
                  [np.sin(body_orientation), np.cos(body_orientation)]])
    accel_local = R.dot(body_data)
    return accel_local
```
The kf_pos has the following state variables (in the local frame): [positionx, positiony, velocityx, velocityy, accelerationx, accelerationy]. We also needed to define the F matrix linking the previous state to the next state such that x_next = F*x_previous. We defined it based on simple movement equation. Since the measurements dont have the same refresh rate we have to update the F matrix at every iteration depending on dt. 
```python
self.kf_pos.F = np.array([[1,0,dt,0,0.5*dt**2,0],
                          [0,1,0,dt,0,0.5*dt**2],
                          [0,0,1,0,dt,0],
                          [0,0,0,1,0,dt],
                          [0,0,0,0,1,0],
                          [0,0,0,0,0,1]])
```
One of the challenge we had to face was to manage to give the measurements with different frequencies. The solution we chose was to declare only two measurement in the declaration of the function and adapt the H matrix (linking the measurement z and the state x such that z = H*x) depending on which type of measurement it was. Which gave those results: 
```python
#for the acceleration
self.kf_pos.H = np.array([[0,0,0,0,1,0],
                         [0,0,0,0,0,1]])
                         #for the acceleration
self.kf_pos.H = np.array([[0,0,1,0,0,0],
                         [0,0,0,1,0,0]])
#for the acceleration
self.kf_pos.H = np.array([[1,0,0,0,0,0],
                         [0,1,0,0,0,0]])
```

The other challenge was to compute the process noise matrix Q. We assumed the noises to be independent so the non-diagonal terms had to be zero. For the diagonal terms we found our answer here: https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python/blob/master/07-Kalman-Filter-Math.ipynb. We don't really understand the 'why' of the math behind it but the kalman converged with this Q matrix so we are happy with it (and it did not work with other tries)
The final implementation of the Q matrix was this: 
```python
#compute Q according to dt 
        self.kf_pos.Q = np.array([
        [1/4 * noise**2 * dt**4, 0, 0, 0, 0, 0],
        [0, 1/4 * noise**2 * dt**4, 0, 0, 0, 0],
        [0, 0, noise**2 * dt**2, 0, 0, 0],
        [0, 0, 0, noise**2 * dt**2, 0, 0],
        [0, 0, 0, 0, noise**2, 0],
        [0, 0, 0, 0, 0, noise**2]])
```
In the end we had to compute the noise of each measurement. We assumed it to be white noise for all measurement and to be equivalent for the x and y part. The process to calculate the noise will be detailled below. For the implementation of the measurement noise matrix we used diagonal matrix with the covariance of the measurement: 

```python
#for acceleration
self.kf_pos.R = (self.ACCEL_NOISE**2)*np.eye(2)
#for velocity
self.kf_pos.R = (self.VEL_NOISE**2)*np.eye(2)
#for position
self.kf_pos.R = (self.POS_NOISE**2)*np.eye(2)
```
We also had to initalize our start variable and the trust we have in it and then we were good to go! Our kalman loop looks like this (here for the acceleration measurement only but the idea doesnt change for others measurements)
```python 
#call this line when you want to update the accelerometer measurement
kf_pos.update_acceleration([accx,accy],current_time)

def update_acceleration(self,data,time):
    
        # calculate time since last measurement and update it
        dt = time- self.time_pos
        self.time_pos = time

        #get current rotation from other kalman
        rotation = self.kf_rot.x.T[0]

        # Transform body frame to local frame
        data = change_frame(data, rotation)

        # Save measurement for plotting
        self.accel_measurement.append(data)

        # Update kf parameters to accept accel measurement
        self.kf_pos.H = np.array([[0,0,0,0,1,0],
                                  [0,0,0,0,0,1]])
        self.kf_pos.R = (self.ACCEL_NOISE**2)*np.eye(2)

        #Update kalman
        self._compute_kf_pos(data,dt)


# Same for velocity, position and acceleration measurement
# Update the F matrix, Q matrix and update the measurement for the position kalman
def _compute_kf_pos(self,data, dt):

    # get accel_noise for process noise matrix
    noise = self.ACCEL_NOISE # most dominant noise

    # compute F according to dt
    self.kf_pos.F = np.array([[1,0,dt,0,0.5*dt**2,0],
                    [0,1,0,dt,0,0.5*dt**2],
                    [0,0,1,0,dt,0],
                    [0,0,0,1,0,dt],
                    [0,0,0,0,1,0],
                    [0,0,0,0,0,1]])
    
    # compute Q according to dt 
    self.kf_pos.Q = np.array([
    [1/4 * noise**2 * dt**4, 0, 0, 0, 0, 0],
    [0, 1/4 * noise**2 * dt**4, 0, 0, 0, 0],
    [0, 0, noise**2 * dt**2, 0, 0, 0],
    [0, 0, 0, noise**2 * dt**2, 0, 0],
    [0, 0, 0, 0, noise**2, 0],
    [0, 0, 0, 0, 0, noise**2]])

    #predict and update kalman
    self.kf_pos.predict()
    self.kf_pos.update(data)
```
## Kalman for rotation 

The kalman for rotation is very similar to the kf_pos except that the measurements differ ( and there is not any frame change). We used as state variable (and measurement also) the heading (from the camera) and the heading rate change (from the wheel speed difference). As the implementation is easier than kf_pos we will not review his full implementation as we can just adapt the steps presented above. 

## Noise processing and data acquisition

For the noise calculation and the measurement equivalence we did the following: 
Position x and position y : Measurement: from the camera. Noise: make the robot stand still with the camera looking at it. Save the camera measurements of the position and extract the covariance. 
Velocity x and Velocity y : Measurement: fix identique speed for both wheel. Measure the time to cover a known distance and deduce a linear speed for all motor input. Always output the average of the two wheels and make it go through the rotation matrix. Noise: We fine tuned it using the position from the camera as a ground truth
Acceleration x and acceleration y: Measurement: from the accelerometer, we changed the value taking the gravity as a reference. Npise: as the accelerometer were giving stable values when the thymio was stopped we use the 9.81/21(equivalent of gravity for accelerometer) as a value. We increased it by 10 later on since it seemed very inaccurate (accelerometers were not precise enough)
Heading change (spin): Measurement: from the wheel. We gave an opposite speed to the 2 wheels than we calculated the time to make 10 turn around itself and calculated the spin in rad/s. From this we computed a linear relation between the spin and the wheel speed difference. Noise: We fined tuned it using the heading from the camera as a ground truth.
Heading: Measurement: from the camera. Noise: make the robot stand still with the camera looking at it. Save the camera measurements of the heading and extract the covariance.

In the end our noises were declared in the kalman class with those values:

``` python
 ACCEL_NOISE = 10*9.81/23
VEL_NOISE = 0.001
ROT_NOISE = 0.005
POS_NOISE = 0.005
SPIN_NOISE = 0.1
```


## Simulation of the kalman

All the simulation of the kalman have been made in kalman.ipynb. We created artificial sensor data and we added white noise to it. We also tested the class we create in test_kalman_class.ipynb (but note that since the noise value are not accurate the result don't fit the curve perfectly but it still is a good example on how to use the Kalman class with rotation and acceleration data). In kalman.ipynb we simulated acceleration measurement and position measurement then stopped position measurement to only rely on accelerometer data. All measurements are in local frame.


<img src="images/kalman_simulation_position.png" alt="Results for the position, the initial position starts at (25,0)" width="400"/>

Legend: Results for the position, the initial position starts at (25,0). We can see that despite having no GPS data the position estimation fits really well the ground truth but drift slowly away from it. The estimation converges quite quickly to the ground truth position in the beginning

<img src="images/kalman_simulation_speed.png" alt="Results for the speed" width="400"/>

Legend: Results for the speed. We can see that it matches the ground truth but it takes somes times to converge when the speed changes. 

<img src="images/kalman_simulation_acceleration.png" alt="Results for the acceleration" width="400"/>

Legend: Results for the acceleration. We can see that we start hitting the limit of the kalman since the shift becomes more important. The results are still accurate


## Real application of the kalman

Now it was time to test our kalman in the robot. We made the robot follow a path, hid the camera for a bit and saw what is gave us. It is important to notice that make the robot working with bluetooth and the sensor values were a bit shifted over time. 

<img src="images/Kalman_setup.png" alt="Kalman results" width="400"/>

Legend: Picture of the kalman path

<img src="images/Kalman_position.png" alt="Kalman results" width="1100"/>

Legend: Kalman position vs Camera measurement. The robot starts on the bottom left. We can see that the kalman estimates the path perfectly when there is camera measurement. When there is not we can see that the inertial navigation does the job even though it drifts a little over time.The results tend to differ from one try to another (espcially when it is connected via cable or bluetooth). In general, the inertial navigation allows the thymio to go across the whole board (long side) with 2-3 turn with a +- good approximation on the position






##### Start the client and wait for a lock

In [ ]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

### Local navigation